In [ ]:
url="http://b2find.eudat.eu/solr/select?echoParams=none&rows=0&wt=json&q=*&facet=true&facet.field=extras_TempCoverageBegin&facet.field=extras_TempCoverageBegin&facet.field=extras_TempCoverageEnd&facet.sort=index"

In [ ]:
import requests

In [ ]:
response=requests.get(url)

In [ ]:
counts=response.json()

In [ ]:
import pandas as pd

In [ ]:
tempbegin = counts["facet_counts"]["facet_fields"]["extras_TempCoverageBegin"]

In [ ]:
countx = tempbegin[1::2]

In [ ]:
seconds = tempbegin[::2]

In [ ]:
df = pd.DataFrame(list(zip(seconds, countx)),
               columns =['Start', 'Number'])
df

In [ ]:
df["Start"]=pd.to_datetime(df["Start"],unit="ms")

In [ ]:
import numpy as np

from bokeh.io import show
from bokeh.layouts import column
from bokeh.models import ColumnDataSource, RangeTool
from bokeh.plotting import figure
from bokeh.sampledata.stocks import AAPL

dates = np.array(AAPL['date'], dtype=np.datetime64)
source = ColumnDataSource(data=dict(date=dates, close=AAPL['adj_close']))
source = ColumnDataSource(df)

p = figure(plot_height=300, plot_width=800, tools="xpan", toolbar_location=None,
           x_axis_type="datetime", x_axis_location="above",
           background_fill_color="#efefef", x_range=(countx[0],countx[10]))

p.line('Start', 'Number', source=source)
p.yaxis.axis_label = 'datasets'

select = figure(title="Drag the middle and edges of the selection box to change the range above",
                plot_height=130, plot_width=800, y_range=p.y_range,
                x_axis_type="datetime", y_axis_type=None,
                tools="", toolbar_location=None, background_fill_color="#efefef")

range_tool = RangeTool(x_range=p.x_range)
range_tool.overlay.fill_color = "navy"
range_tool.overlay.fill_alpha = 0.2

select.line('Start', 'Number', source=source)
select.ygrid.grid_line_color = None
select.add_tools(range_tool)
select.toolbar.active_multi = range_tool

show(column(p, select))
